Сегодня не будет информации, напрямую связанной с машинным обучением. Сегодня будут обсуждаться полезные моменты вокруг.

# Bash & shell scripting

Bash - наиболее популярная командная оболочка в UNIX-like система. Когда вы пользуетесь терминалом на своем компьютере, то вы почти наверное используете именно Bash.

В баш встроено огромное количество функциональности, которая не ограничивается лишь запуском программ.

In [2]:
! mkdir -p bash

In [5]:
%%writefile bash/program-example.sh

function pow2 {
    local argument=$1
    if [[ argument -eq 1 ]]; then
        echo 2
    elif [[ $((argument % 2)) -eq 0 ]]; then
        local new_pow=$((argument / 2))
        local sqrt_result=$(pow2 $new_pow)
        echo $((sqrt_result * sqrt_result))
    else
        local new_pow=$((argument - 1))
        local halfed_result=$(pow2 $new_pow)
        echo $((halfed_result * 2))
    fi
}

word=$1
power=$2

limit=$(pow2 $power)

for ((i = 0 ; i < limit ; i++)); do
  echo $i $word
done

Overwriting bash/program-example.sh


In [6]:
! bash bash/program-example.sh hello 4

0 hello
1 hello
2 hello
3 hello
4 hello
5 hello
6 hello
7 hello
8 hello
9 hello
10 hello
11 hello
12 hello
13 hello
14 hello
15 hello


В примере выше, скрипт на чистом баше пишет указанное слово 2^N раз. 

Можно заметить, что несмотря на теоретическую возможность реализации любого алгоритма на этом языке, он не слишком удобен для подобных задач. Поэтому останавливаться долго на том, как лучше всего программировать алгоритмы на баш не будем, а посмотрим на более практичные и полезные способности.

#### Флаги исполнения
При запуске башового скрипта бывает полезно указывать флаги исполнения. Это может помочь при запуске как своих, так и сторонних скриптов.

* `-x` Печатает процесс исполнения скрипта. Полезно при отладке.
* `-n` Только проверяет скрипт на синтаксическую корректность - не запускает.
* `-i` Запускает в интерактивном режиме. Есть возможность пообщаться с рантайм-окружением скрипта.
* `-e` Уронить скрипт после первой неудачной команды. Без указания скрипт попытается запустить все команды, которые в нем прописаны, что может привести к еще более печальным последствиям.

Более полный список флагов можно найти здесь - https://www.tldp.org/LDP/abs/html/options.html

In [8]:
! bash -x bash/program-example.sh hello 2

+ word=hello
+ power=2
++ pow2 2
++ local argument=2
++ [[ argument -eq 1 ]]
++ [[ 0 -eq 0 ]]
++ local new_pow=1
+++ pow2 1
+++ local argument=1
+++ [[ argument -eq 1 ]]
+++ echo 2
++ local sqrt_result=2
++ echo 4
+ limit=4
+ (( i = 0  ))
+ (( i < limit  ))
+ echo 0 hello
0 hello
+ (( i++ ))
+ (( i < limit  ))
+ echo 1 hello
1 hello
+ (( i++ ))
+ (( i < limit  ))
+ echo 2 hello
2 hello
+ (( i++ ))
+ (( i < limit  ))
+ echo 3 hello
3 hello
+ (( i++ ))
+ (( i < limit  ))


In [11]:
! bash -n bash/program-example.sh hello 2

In [49]:
%%writefile bash/broken-curl.sh

curl http://non.existing.domain/file.txt > file.txt
    
echo -n "Number of lines in file is equal to "
cat file.txt | wc -l

Overwriting bash/broken-curl.sh


In [50]:
! bash bash/broken-curl.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: non.existing.domain
Number of lines in file is equal to 0


In [51]:
! bash -e bash/broken-curl.sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: non.existing.domain


Полезно также указывать их в самом скрипте

In [52]:
%%writefile bash/broken-curl-flag.sh

set -xe

curl http://non.existing.domain/file.txt > file.txt
    
echo -n "Number of lines in file is equal to "
cat file.txt | wc -l

Writing bash/broken-curl-flag.sh


In [53]:
! bash bash/broken-curl-flag.sh

+ curl http://non.existing.domain/file.txt
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: non.existing.domain


#### Перенаправления и композиции

Башовые команды работают преимущественно с потоками ввода и вывода. В самом первом примере можно заметить, что внутри функции используется не return а echo. Это связанно с тем, что возвращенным значением какой-либо команды является exit code этой команды, который используется для определения успешности завершения. Сами результаты работы внутри баша текуч через stdin\stdout.

* Вывод stdout можно перенаправить на запись в файл

In [54]:
! echo hello > hello.txt

In [55]:
! cat hello.txt

hello


In [56]:
! echo hello > hello.txt

In [57]:
! cat hello.txt

hello


In [58]:
! echo hello >> hello.txt

In [59]:
! cat hello.txt

hello
hello


* И наоборот содержание файла можно подавать на вход stdin 

In [66]:
! python3 -c "import sys; print(list(sys.stdin))" < hello.txt

['hello\n', 'hello\n']


Если хочется направить на stdin несколько строк, но при этом не из файла, то можно воспользваться оператором `<<`. Для него нужно отдельно указать маркер начала и конца данных.

In [87]:
%%bash

python3 -c "import sys; print(list(sys.stdin))" <<END
hello
world
END

['hello\n', 'world\n']


Или аналогично с однострочной строкой

In [79]:
%%writefile bash/redirect-string-to-stdin.sh

python3 -c "import sys; print(list(sys.stdin))" <<< "hello world"

Writing bash/redirect-string-to-stdin.sh


In [80]:
! bash bash/redirect-string-to-stdin.sh

['hello world\n']


Комбинирование этих возможностей может позволить решить некоторые задачи. Например генерировать какой-то файл на лету

In [86]:
%%writefile bash/generate-file.sh

argument=$1

cat > file.txt <<END
This file was automatically generated with argument $argument
END

Overwriting bash/generate-file.sh


In [87]:
! bash bash/generate-file.sh PUPA

In [88]:
! cat file.txt

This file was automatically generated with argument PUPA


Или например решать проблема с интерактивными программами

In [95]:
%%writefile bash/interactive.py

x = input('Do you want to print word `hello`? (Y/n)')
print('hello' if x == 'y' else 'something else')

Overwriting bash/interactive.py


In [96]:
%%writefile bash/non-interactive.sh

python3 bash/interactive.py <<< "y"
python3 bash/interactive.py <<< "n"

Overwriting bash/non-interactive.sh


In [97]:
! bash bash/non-interactive.sh

Do you want to print word `hello`? (Y/n)hello
Do you want to print word `hello`? (Y/n)something else


Отдельно можно отметить, что специально для вопросов yes\no была создана программа `yes`. Единственное, что она делает - это без остановки печатает `y`.

Для этого можно перенаправить вывод из команды в stdin. Для этог также есть свой оператор, однако он работает не так просто, как можно было бы подумать с первого раза. 

Команда вида `command1 <(command2)` работает следующим образом
* Создается временный сокет (читай - файл) и в него потоком льется stdout от команды command2
* Путь до этого виртуального файла передается в command1 через агрументы командной строки

In [88]:
%%bash

echo <(echo foo)

/dev/fd/63


In [89]:
%%bash

cat <(echo bar)
python3 -c "import sys; print(open(sys.argv[1], 'r').read())" <(echo hello)

bar
hello



Таким образом, комбинируюя операторы `<` и `<()` можно перенаправлять вывод одной команды в ввод другой.

In [90]:
%%bash

cat < <(ls -l *.ipynb)

python3 bash/interactive.py < <(yes)

-rw-rw-r-- 1 1001 1002   21509 Feb  2 20:17 3. Spark. DRAFT.ipynb
-rw-r--r-- 1 root root   40717 Feb 17 22:27 5. Streaming algorithms.ipynb
-rw-rw-r-- 1 1001 1002  320455 Mar  8 17:16 6. KNN problem.ipynb
-rw-r--r-- 1 root root   23627 Feb 28 21:09 7.1 KubeflowKFP.ipynb
-rw-r--r-- 1 root root   37269 Mar  4 08:47 7. Azure ML.ipynb
-rw-r--r-- 1 root root     555 Mar  3 23:10 7. Azure ML second.ipynb
-rw-r--r-- 1 root root  132192 Mar  3 19:56 7. Kubeflow.ipynb
-rw-r--r-- 1 1001 1002   49064 Mar 10 18:17 8. Distributed Neural Networks.ipynb
-rw-r--r-- 1 root root   45848 Mar 10 21:05 9. Additional notes.ipynb
-rw-r--r-- 1 root root    5917 Feb 17 19:23 Azure cluster_1.1.ipynb
-rw-r--r-- 1 root root   13073 Jan 19 19:20 MapReduce_1.2.ipynb
-rw-r--r-- 1 root root   16217 Jan 19 19:23 MapReduce_2.ipynb
-rw-rw-r-- 1 1001 1002  106413 Feb 11 20:29 Spark-SQL-VW.ipynb
-rw-r--r-- 1 root root   48112 Feb 20 22:48 task1.ipynb
-rw-r--r-- 1 root root    1025 Mar  8 12:25 Untitled1.ipynb
-rw-r--r-- 1

In [91]:
%%bash
diff <(ls /bin) <(ls /usr/bin)

1,71c1,593
< bash
< bunzip2
< bzcat
< bzcmp
< bzdiff
< bzegrep
< bzexe
< bzfgrep
< bzgrep
< bzip2
< bzip2recover
< bzless
< bzmore
< cat
< chgrp
< chmod
< chown
< cp
< dash
< date
< dd
< df
< dir
< dmesg
< dnsdomainname
< domainname
< echo
< egrep
< false
< fgrep
< findmnt
< grep
< gunzip
< gzexe
< gzip
< hostname
< kill
< ln
< login
< ls
< lsblk
< mkdir
< mknod
< mktemp
< more
< mount
< mountpoint
< mv
< nano
< nc
< nc.traditional
< netcat
< nisdomainname
< pidof
< ps
< pwd
< rbash
< readlink
< rm
< rmdir
< rnano
< run-parts
< sed
< sh
< sh.distrib
< sleep
< stty
< su
< sync
< tar
< tempfile
---
> [
> 2to3-2.7
> addpart
> addr2line
> afm2pl
> afm2tfm
> aleph
> allcm
> allec
> allneeded
> apt
> apt-cache
> apt-cdrom
> apt-config
> apt-get
> apt-key
> apt-mark
> ar
> arch
> as
> aspell
> aspell-import
> authorindex
> autosp
> awk
> az
> b2sum
> base32
> base64
> basename
> bashbug
> bibtex
> bibtex8
> bibtex.original
> bibtexu
> browse
> c++
> c89
> c89-gcc
> c99
> c99-gcc
> cachepic
> 

CalledProcessError: Command 'b'diff <(ls /bin) <(ls /usr/bin)\n'' returned non-zero exit status 1.

Идея соединять процессы через stdin\stdout очень популярна и для этого есть более удобный интерфейс - pipes.

Все команды, соединенные через `|` запускаются одновременно и общаются друг с другом через stdin\stdout

In [92]:
%%bash

echo hello | cat
ls -l | cat | wc -l | python3 -c "print(int(input()) * 2)"

hello
140


In [61]:
! yes | python3 bash/interactive.py

Do you want to print word `hello`? (Y/n)hello
yes: standard output: Broken pipe


Помимо файлов и потоков ввода\вывода, bash умеет возврашать результат работы команды напрямую в скрипт. Это можно использовать например чтобы перенаправлять в аргументы командной строки.

In [62]:
! echo $(echo hello)

hello


In [76]:
! echo $(ls bash/* | head -n 3)

bash/broken-curl-flag.sh bash/broken-curl.sh bash/diff-in-dir-files.sh


In [77]:
! cat $(ls bash/* | head -n 3)


set -xe

curl http://non.existing.domain/file.txt > file.txt
    
echo -n "Number of lines in file is equal to "
cat file.txt | wc -l

curl http://non.existing.domain/file.txt > file.txt
    
echo -n "Number of lines in file is equal to "
cat file.txt | wc -l

diff <(ls /bin) <(ls /usr/bin)


In [100]:
%%bash

echo "There are $(cat bash/non-interactive.sh | wc -l) lines in file"

There are 3 lines in file


Отдельно стоит отметить, что ординарные скобки обрабатывают результат как просто строку, в то время как двойные скобки позволяют работать с арифметическими выражениями

In [301]:
%%bash

hello=$(ls -l | wc -l)
echo $hello
echo $(( $hello + 1 ))

echo $(( $(ls -l | wc -l) / 3 ))

73
74
24


#### Построение пайплайнов

Мы уже видели, каким образом можно запускать задачи параллельно, связывая их потоки ввода\вывода.
Для полноты картины нужно научиться составлять последовательные пайплайны

* `&&` Последующие команды будут выполнены, только если успешно завершилась предыдущие

In [114]:
%%bash

echo hello && echo world

hello
world


In [116]:
! rm non-exist-dir && echo world

rm: cannot remove 'non-exist-dir': No such file or directory


* `||` Последующие команды будут выполнены, только если неуспешно завершились предыдущие

In [118]:
! echo hello || echo world

hello


In [119]:
! rm non-exists-dir || echo world

rm: cannot remove 'non-exists-dir': No such file or directory
world


Команду можно изолизовать в подкоманду, используя скобки. Таким образом можно например обходить ситуации, когда не так важно, успешно завершилась команда или нет.

In [120]:
! (rm non-exists-dir || exit 0) && echo world

rm: cannot remove 'non-exists-dir': No such file or directory
world


#### Синтаксический сахар

Bash умеет распупочивать команды для более короткой записи.

In [102]:
%%bash

echo letter_{A,B,C}
echo number_{1..5}

letter_A letter_B letter_C
number_1 number_2 number_3 number_4 number_5


In [104]:
%%bash

ls traffic_{feb_june,june_sep}/* | wc -l

898


In [113]:
%%bash

echo {A,B,C}_{1..5}/{dir_1,dir_2}/{subdir_{1..3}}\\n

A_1/dir_1/{subdir_1}\n A_1/dir_1/{subdir_2}\n A_1/dir_1/{subdir_3}\n A_1/dir_2/{subdir_1}\n A_1/dir_2/{subdir_2}\n A_1/dir_2/{subdir_3}\n A_2/dir_1/{subdir_1}\n A_2/dir_1/{subdir_2}\n A_2/dir_1/{subdir_3}\n A_2/dir_2/{subdir_1}\n A_2/dir_2/{subdir_2}\n A_2/dir_2/{subdir_3}\n A_3/dir_1/{subdir_1}\n A_3/dir_1/{subdir_2}\n A_3/dir_1/{subdir_3}\n A_3/dir_2/{subdir_1}\n A_3/dir_2/{subdir_2}\n A_3/dir_2/{subdir_3}\n A_4/dir_1/{subdir_1}\n A_4/dir_1/{subdir_2}\n A_4/dir_1/{subdir_3}\n A_4/dir_2/{subdir_1}\n A_4/dir_2/{subdir_2}\n A_4/dir_2/{subdir_3}\n A_5/dir_1/{subdir_1}\n A_5/dir_1/{subdir_2}\n A_5/dir_1/{subdir_3}\n A_5/dir_2/{subdir_1}\n A_5/dir_2/{subdir_2}\n A_5/dir_2/{subdir_3}\n B_1/dir_1/{subdir_1}\n B_1/dir_1/{subdir_2}\n B_1/dir_1/{subdir_3}\n B_1/dir_2/{subdir_1}\n B_1/dir_2/{subdir_2}\n B_1/dir_2/{subdir_3}\n B_2/dir_1/{subdir_1}\n B_2/dir_1/{subdir_2}\n B_2/dir_1/{subdir_3}\n B_2/dir_2/{subdir_1}\n B_2/dir_2/{subdir_2}\n B_2/dir_2/{subdir_3}\n B_3/dir_1/{subdir_1}\n B_3/dir_1/{

Bash умеет запускать предыдущую команду через `!!`

In [124]:
$ echo hello
$ !!

hello


Через установку `alias` можно самому добавлять команды для новых символов

In [ ]:
$ alias check='ls -l /bin | wc -l'
$ check

#### Полезные программы

Сами программы не являются часть баша, однако их можно использовать гораздо эффективнее, комбинируя их работу с башовыми трюками

In [128]:
# head читает определенное количество данных с начала файла
! head bash/interactive.py
! head -n 2 bash/interactive.py  # 2 строки
! head -c 10 bash/interactive.py # 10 байт


x = input('Do you want to print word `hello`? (Y/n)')
print('hello' if x == 'y' else 'something else')

x = input('Do you want to print word `hello`? (Y/n)')

x = input

In [129]:
# tail делает тоже самое, что и head, но с конца
! tail bash/interactive.py
! tail -n 2 bash/interactive.py # 2 строки
! tail -n +2 bash/interactive.py # все строки после 2


x = input('Do you want to print word `hello`? (Y/n)')
print('hello' if x == 'y' else 'something else')
x = input('Do you want to print word `hello`? (Y/n)')
print('hello' if x == 'y' else 'something else')
x = input('Do you want to print word `hello`? (Y/n)')
print('hello' if x == 'y' else 'something else')


`tail -f <file>` позволит следить за изменениями в файле

In [131]:
%%bash 

cat > example.txt <<END
3
5
1
2
6
5
9
4
5
6
7
3
2
1
5
6
END

In [132]:
# sort сортирует содержимое входа
! sort example.txt

1
1
2
2
3
3
4
5
5
5
5
6
6
6
7
9


In [133]:
! cat example.txt | sort

1
1
2
2
3
3
4
5
5
5
5
6
6
6
7
9


In [138]:
# uniq отбирает только уникальные значения, но ему на вход необходимо подавать уже отсортированный массив
! uniq example.txt

3
5
1
2
6
5
9
4
5
6
7
3
2
1
5
6


In [139]:
! cat example.txt | sort | uniq

1
2
3
4
5
6
7
9


In [140]:
# cut парсит строки с разделителем, что позволяет легко манипулировать различными типами данных
! echo "1,2,3,4" | cut -d',' -f2

2


In [145]:
! cat traffic_feb_june/trafficData158324.csv | cut -d',' -f6,7 | head

2014-02-13T11:30:00,7
2014-02-13T11:35:00,5
2014-02-13T11:40:00,6
2014-02-13T11:45:00,3
2014-02-13T11:50:00,6
2014-02-13T11:55:00,9
2014-02-13T12:00:00,11
2014-02-13T12:05:00,8
2014-02-13T12:10:00,10
2014-02-13T12:15:00,12
cut: write error: Broken pipe


In [159]:
! cat traffic_feb_june/trafficData158324.csv | cut -d',' -f8 | sort | uniq | head

10000005
10000454
10000903
10001352
10001801
10002250
10002699
10003148
10003597
10004046
uniq: write error: Broken pipe


In [162]:
# wc - word count - подсчитывает количество симолов на входе. Опция -l позволяет считать количество строк
! cat example.txt | wc -l

16


In [163]:
# Подсчет уникальных элементов 
! cat traffic_feb_june/trafficData158324.csv | cut -d',' -f8 | sort | uniq | wc -l

32059


In [164]:
# grep - Позволяет обрабатывать входы регулярными выражениями.
! cat example.txt | grep 2

2
2


In [165]:
! cat example.txt | grep -e "[0-9]*"

3
5
1
2
6
5
9
4
5
6
7
3
2
1
5
6


In [169]:
! cat 9.\ Additional\ notes.ipynb | grep -e 'uniq'

      "> uniq\n",
    "# uniq отбирает только уникальные значения, но ему на вход необходимо подавать уже отсортированный массив\n",
    "! uniq example.txt"
    "! cat example.txt | sort | uniq"
      "uniq: write error: Broken pipe\r\n"
    "! cat traffic_feb_june/trafficData158324.csv | cut -d',' -f8 | sort | uniq | head"
    "! cat traffic_feb_june/trafficData158324.csv | cut -d',' -f8 | sort | uniq | wc -l"
    "! cat 9.\\ Additional\\ notes.ipynb | grep -e '{uniq,cut}'"


In [182]:
! cat $(ls traffic_feb_june/* | grep -e ".*\.csv$") | grep -e "^OK" | cut -d',' -f8 | sort | uniq | wc -l

13573137


In [185]:
# Также сам grep умеет искать рекурсивно в директории
! grep -R -e 'cut\ ' *.ipynb

9. Additional notes.ipynb:    "# cut парсит строки с разделителем, что позволяет легко манипулировать различными типами данных\n",
9. Additional notes.ipynb:    "! echo \"1,2,3,4\" | cut -d',' -f2"
9. Additional notes.ipynb:    "! cat traffic_feb_june/trafficData158324.csv | cut -d',' -f6,7 | head"
9. Additional notes.ipynb:    "! cat traffic_feb_june/trafficData158324.csv | cut -d',' -f8 | sort | uniq | head"
9. Additional notes.ipynb:    "! cat traffic_feb_june/trafficData158324.csv | cut -d',' -f8 | sort | uniq | wc -l"
9. Additional notes.ipynb:      "    \"! cat traffic_feb_june/trafficData158324.csv | cut -d',' -f8 | sort | uniq | head\"\r\n",
9. Additional notes.ipynb:      "    \"! cat traffic_feb_june/trafficData158324.csv | cut -d',' -f8 | sort | uniq | wc -l\"\r\n",
9. Additional notes.ipynb:    "! cat $(ls traffic_feb_june/* | grep -e \".*\\.csv$\") | grep -e \"^OK\" | cut -d',' -f8 | sort | uniq | wc -l"


In [203]:
%%bash

# sed - stream editor. Позволяет изменять входящий поток. У него есть ряд команд -
# p = print, d = delete, s = substitude, 

cat example.txt | sort

1
1
2
2
3
3
4
5
5
5
5
6
6
6
7
9


In [226]:
%%bash

cat example.txt | sort | sed -n '3'p  # третий элемент
echo "---"
cat example.txt | sort | sed -n '0~3'p  # Каждый третий начиная с нулевого
echo "---"
cat example.txt | sort | sed -n "3,5"p # C 3 по 5
echo "---"
cat example.txt | sort | sed -n "6,$"p # c 6 до конца файла
echo '---'
cat example.txt | sort | sed -n '/5/'p # вместо номера можно указывать regex. этот пример почти такой же как и grep
echo '---'
cat example.txt | sort | sed -n "/5/,/7/"p # можно комбинироватьк как и числа

2
---
2
3
5
6
7
---
2
2
3
---
3
4
5
5
5
5
6
6
6
7
9
---
5
5
5
5
---
5
5
5
5
6
6
6
7


In [227]:
%%bash
# команда d - инвертированная от p - удаляет указанные, а значит печатает те, которые вне диапазона

cat example.txt | sort | sed -e '3'd  # третий элемент
echo "---"
cat example.txt | sort | sed -e '0~3'd  # Каждый третий начиная с нулевого
echo "---"
cat example.txt | sort | sed -e "3,5"d # C 3 по 5
echo "---"
cat example.txt | sort | sed -e "6,$"d # c 6 до конца файла
echo '---'
cat example.txt | sort | sed -e '/5/'d # вместо номера можно указывать regex. этот пример почти такой же как и grep
echo '---'
cat example.txt | sort | sed -e "/5/,/7/"d # можно комбинироватьк как и числа

1
1
2
3
3
4
5
5
5
5
6
6
6
7
9
---
1
1
2
3
4
5
5
5
6
6
9
---
1
1
3
4
5
5
5
5
6
6
6
7
9
---
1
1
2
2
3
---
1
1
2
2
3
3
4
6
6
6
7
9
---
1
1
2
2
3
3
4
9


In [228]:
%%writefile script-example.sh

cat example.txt | sort | sed -e '3'd  # третий элемент
echo "---"
cat example.txt | sort | sed -e '0~3'd  # Каждый третий начиная с нулевого
echo "---"
cat example.txt | sort | sed -e "3,5"d # C 3 по 5
echo "---"
cat example.txt | sort | sed -e "6,$"d # c 6 до конца файла
echo '---'
cat example.txt | sort | sed -e '/5/'d # вместо номера можно указывать regex. этот пример почти такой же как и grep
echo '---'
cat example.txt | sort | sed -e "/5/,/7/"d # можно комбинироватьк как и числа

Writing script-example.sh


In [237]:
%%bash

# можно заменять regex на значение

cat script-example.sh | sed -e "s/sort/sort | uniq/"
cat script-example.sh | sed -e "s/sort/sort | uniq/" | bash


cat example.txt | sort | uniq | sed -e '3'd  # третий элемент
echo "---"
cat example.txt | sort | uniq | sed -e '0~3'd  # Каждый третий начиная с нулевого
echo "---"
cat example.txt | sort | uniq | sed -e "3,5"d # C 3 по 5
echo "---"
cat example.txt | sort | uniq | sed -e "6,$"d # c 6 до конца файла
echo '---'
cat example.txt | sort | uniq | sed -e '/5/'d # вместо номера можно указывать regex. этот пример почти такой же как и grep
echo '---'
cat example.txt | sort | uniq | sed -e "/5/,/7/"d # можно комбинироватьк как и числа
1
2
4
5
6
7
9
---
1
2
4
5
7
9
---
1
2
6
7
9
---
1
2
3
4
5
---
1
2
3
4
6
7
9
---
1
2
3
4
9


In [259]:
%%bash

# awk - самый серьезный инструмент для обработки данных. Он имеет свой отделный язык написания запросов
echo pupa lupa | awk '{print $1}'
echo pupa lupa | awk '{print $2}'

echo '---'

cat traffic_feb_june/trafficData158324.csv | awk -F, '{print "Code = " $1 " Report = " $8}' | head

echo '---'

cat traffic_feb_june/trafficData158324.csv | awk -F, 'BEGIN {print "code,report"} {print $1 "," $8}' | head

echo '---'

cat traffic_feb_june/trafficData158324.csv | awk -F, '{print $8, " | This is line number " FNR}' | head

echo '---'

cat traffic_feb_june/trafficData158324.csv | awk -F, '{if ($8 % 2 == 0) print $8, $6}' | head

pupa
lupa
---
Code = OK Report = 190000
Code = OK Report = 190449
Code = OK Report = 190898
Code = OK Report = 191347
Code = OK Report = 191796
Code = OK Report = 192245
Code = OK Report = 192694
Code = OK Report = 193143
Code = OK Report = 193592
Code = OK Report = 194041
---
code,report
OK,190000
OK,190449
OK,190898
OK,191347
OK,191796
OK,192245
OK,192694
OK,193143
OK,193592
---
190000  | This is line number 1
190449  | This is line number 2
190898  | This is line number 3
191347  | This is line number 4
191796  | This is line number 5
192245  | This is line number 6
192694  | This is line number 7
193143  | This is line number 8
193592  | This is line number 9
194041  | This is line number 10
---
190000 2014-02-13T11:30:00
190898 2014-02-13T11:40:00
191796 2014-02-13T11:50:00
192694 2014-02-13T12:00:00
193592 2014-02-13T12:10:00
194490 2014-02-13T12:20:00
195388 2014-02-13T12:30:00
196286 2014-02-13T12:40:00
197184 2014-02-13T12:50:00
198082 2014-02-13T13:00:00


In [261]:
! apt-get update -y && apt-get install jq -y

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease                
Hit:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease              
Hit:4 http://security.ubuntu.com/ubuntu bionic-security InRelease              
Hit:5 https://packages.microsoft.com/repos/azure-cli bionic InRelease   
Reading package lists... Done                      
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libjq1 libonig4
The following NEW packages will be installed:
  jq libjq1 libonig4
0 upgraded, 3 newly installed, 0 to remove and 77 not upgraded.
Need to get 276 kB of archives.
After this operation, 930 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libonig4 amd64 6.7.0-1 [119 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libjq1 amd64 1.5+dfsg-2 [11

In [266]:
! wget https://raw.githubusercontent.com/prust/wikipedia-movie-data/master/movies.json

--2020-03-10 23:17:08--  https://raw.githubusercontent.com/prust/wikipedia-movie-data/master/movies.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3386161 (3.2M) [text/plain]
Saving to: ‘movies.json’

movies.json         100%[===================>]   3.23M  8.61MB/s    in 0.4s    

2020-03-10 23:17:10 (8.61 MB/s) - ‘movies.json’ saved [3386161/3386161]



In [281]:
%%bash

# По умолчанию jq не входит в набор утилит, но при этом это очень полезная тулза

cat movies.json | head -c 1000

echo '---'

cat movies.json | jq '.' | head -n 30
echo '---'

cat movies.json | jq '.[0:2]'
echo '---'

cat movies.json | jq '.[] | .title' | head
echo '---'
cat movies.json | jq 'map(.year-1900)' | head

[{"title":"After Dark in Central Park","year":1900,"cast":[],"genres":[]},{"title":"Boarding School Girls' Pajama Parade","year":1900,"cast":[],"genres":[]},{"title":"Buffalo Bill's Wild West Parad","year":1900,"cast":[],"genres":[]},{"title":"Caught","year":1900,"cast":[],"genres":[]},{"title":"Clowns Spinning Hats","year":1900,"cast":[],"genres":[]},{"title":"Capture of Boer Battery by British","year":1900,"cast":[],"genres":["Short","Documentary"]},{"title":"The Enchanted Drawing","year":1900,"cast":[],"genres":[]},{"title":"Feeding Sea Lions","year":1900,"cast":["Paul Boyton"],"genres":[]},{"title":"How to Make a Fat Wife Out of Two Lean Ones","year":1900,"cast":[],"genres":["Comedy"]},{"title":"New Life Rescue","year":1900,"cast":[],"genres":[]},{"title":"New Morning Bath","year":1900,"cast":[],"genres":[]},{"title":"Searching Ruins on Broadway, Galveston, for Dead Bodies","year":1900,"cast":[],"genres":[]},{"title":"The Tribulations of an Amateur Photographer","year":1900,"cast":

In [295]:
%%bash

cat movies.json | jq '.[] | .title' | sed -e 's/\"//g' | sed -e 's/\s/\n/g' | sort | sed -n '200,220p'
echo '------'
cat movies.json | jq '.[] | .title' | sed -e 's/\"//g' | sed -e 's/\s/\n/g' | sort | awk '{words[$1] += 1} END {for (word in words) {print word, words[word]}}' | head

109
10th
10th
11
11
11
1-1000
110th
11-11-11
11:14
1119
113
1138
11/9
11th
12
12
12
12
12
12
------
Royale 3
Remains 3
Planets 1
Juliet, 1
Ticklish 1
Piper's 1
Notebook 1
and... 1
Willie 7
Troubles 6


__Задача__ (все задачи опциональные - за дополнительный балл)
* Для датасета из 5 семинара про стриминговые алгоритмы
  * Подсчитать среднее для 7 колонки (при нумерации с 1) (количество машин) на bash
  * Подсчитать медиану для 7 колонки (при нумерации с 1) (количество машин) на bash
  * Сравнить время работы алгоритма на bash и алгоритмов на python из 5 семинара
* Для датасета из 1 семинара 
  * Подсчитать топ 20 самых встречаемых слов на bash
  * Сравнить время работы алгоритма на bash и алгоритма на python из 1 семинара